# Memory

Describing memory and getting it synthesized correctly is cumbersome.
Therefore, this will only demonstrate a simple approach to generate memory.
Using [Signal arrays](arrays.ipynb), a simple `r1w1` simplex dual port memory can be inferred to HDL.
This modification enables shared variable output for VHDL to support true dual port behaviour.
The general strategy with the MyIRL synthesis is to use a blackbox element (rely on the library)

In [1]:
from myirl import *
import random

## Dual port memories

We can not use a `SigArray` for the RAM, because a true dual port RAM would provoke unresolved multiple drivers.
Therefore we hack the `myirl.lists.SigArray` class by deriving and choosing a different element in return of the `__getitem__` member which uses a shared variable.

**Note** Not working in VHDL-2008

In [2]:
from myirl import lists
from myirl.kernel import extensions, utils

from myirl import targets

class ArrayElem(lists.SigIndexed):
    decl_type_vhdl = "shared variable"

    def set(self, val):
        w = self.size()
        return CellAssign(self.seq, self.index, val, w)

class CellAssign:  # extensions.ElemAssign
    def __init__(self, parent, portion, val, width):
        self._vref = parent
        self.portion = portion
        self.value = val
        self.width = width
        
    def emit(self, context):
        tgt = context.target
        n, v = self._vref.identifier, self.value
        p = self.portion
        sz = self.width
        if tgt.lang == 'VHDL':
            context.output("%s(to_integer(%s)) := %s;\n" % (n, p, base.convert(v, tgt, sz)))        
        elif tgt.lang == 'Verilog':
            context.output("%s[%s] <= %s;\n" % (n, p, base.convert(v, tgt, sz)))           
        else:
            raise TypeError("Unsupported target %s" % type(tgt))


    def get_sources(self, srcs):
        if isinstance(self.value, base.Sig):
            self.value.get_sources(srcs)
    
    def get_drivers(self, drvs):
        pass
    

class RamBuffer(lists.SigArray):    
    init = True
    sigclass = targets.Verilog.REG
    def __getitem__(self, item):
        if isinstance(item, (base.Sig, int)):
            # We can not just return self.val[item], as the iterator
            # has not initialized yet.
            return ArrayElem(self, item)
        else:
            raise TypeError("Multi item slicing of iterator not supported")

Then define a RAM port (legacy class constructs will suffice) and the actual RAM implementation:

In [3]:
@container()
class RamPort:
    _inputs = ['we', 'wa', 'ra', 'wd']
    _outputs = ['rd']
    _other = ['clk']
    def __init__(self, AWIDTH, DWIDTH):
        self.clk = ClkSignal()
        self.we = Signal(bool())
        self.ra, self.wa = [ Signal(intbv()[AWIDTH:]) for i in range(2) ]
        self.rd, self.wd = [ Signal(intbv()[DWIDTH:]) for i in range(2) ] 
        
@block
def tdp_ram(pa : RamPort, pb : RamPort, INITDATA : PassThrough(list)):
    inst = []
 
    def gen_logic(p, i):
        "Generate port mechanics inline"
        
        rd = p.rd.clone("rd%d" % i)
        
        @genprocess(p.clk, EDGE=p.clk.POS)
        def proc():
            yield [
                proc.If(p.we == True).Then(
                    buf[p.wa].set(p.wd)
                ),
                rd.set(buf[p.ra])
            ]
        proc.rename("proc%d" % i)
        wirings = [
            p.rd.wireup(rd)
        ]
        return proc, wirings

    buf = RamBuffer(INITDATA)
    
    for i, p in enumerate([pa, pb]):
        inst += (gen_logic(p, i))
    
    return instances()

### Basic checks

Verify we can evaluate the content:

In [4]:
a = RamBuffer([intbv(i + 0x80)[8:] for i in range(200) ])
b = Signal(intbv(10)[5:])
a[b].evaluate()

intbv(138)

In [5]:
a[0].wire

intbv(128)

In [6]:
from myirl import targets
from myirl.test.common_test import run_ghdl, run_icarus

random.seed(0)

def test_vhdl():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.VHDL, elab_all = True)
    run_ghdl(f, inst, std = "93", debug = True) # Note we run with std '93'
    return f

In [7]:
f = test_vhdl()

 Writing 'tdp_ram' to file /tmp/myirl_tdp_ram_q2vdmwwd/tdp_ram.vhdl 
 Creating library file module_defs.vhdl 
WORK DIR of instance [Instance tdp_ram I/F: [// ID: tdp_ram_0 to tdp_ram]] /tmp/myirl_tdp_ram_q2vdmwwd/
==== COSIM stderr ====
/tmp/myirl_tdp_ram_q2vdmwwd/tdp_ram.vhdl:46:17: signal "v_ce16" is not a variable to be assigned
/tmp/myirl_tdp_ram_q2vdmwwd/tdp_ram.vhdl:58:17: signal "v_ce16" is not a variable to be assigned



In [8]:
! grep -A 20 MyIRL {f[0]}

## Verilog variant

The same design translated to Verilog, run through the icarus simulator.

In [9]:
def test_verilog():
    RAM_CONTENT = [ intbv(random.randint(0, 2 ** 9))[8:] for i in range(2 ** 9) ]
    pa, pb = [ RamPort(AWIDTH=9, DWIDTH=8) for i in range(2) ]
    inst = tdp_ram(pa, pb, RAM_CONTENT)
    
    f = inst.elab(targets.Verilog)
    run_icarus(f, inst, debug = True)
    return f

In [10]:
f = test_verilog()

 Module tdp_ram: Existing instance tdp_ram, rename to tdp_ram_1 
 Writing 'tdp_ram_1' to file /tmp/myirl_tdp_ram_4ggkar1m/tdp_ram_1.v 
DEBUG: Source 'v_6288' is logic: <class '__main__.RamBuffer'>
DEBUG Fallback wire for pa_clk
DEBUG Fallback wire for pa_wa
DEBUG Fallback wire for pa_wd
DEBUG Fallback wire for pb_clk
DEBUG Fallback wire for pb_wa
DEBUG Fallback wire for pb_wd
ICARUS FILES ['/tmp/myirl_tdp_ram_4ggkar1m/tdp_ram_1.v']


In [11]:
!cat {f[0]}

// File generated from source:
//     /tmp/ipykernel_40938/1352711214.py
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture myIRL

module tdp_ram_1
    (
        input wire  pa_clk,
        input wire  pa_we,
        input wire [8:0] pa_ra,
        input wire [8:0] pa_wa,
        output wire [7:0] pa_rd,
        input wire [7:0] pa_wd,
        input wire  pb_clk,
        input wire  pb_we,
        input wire [8:0] pb_ra,
        input wire [8:0] pb_wa,
        output wire [7:0] pb_rd,
        input wire [7:0] pb_wd
    );
    // Local type declarations
    // Signal declarations
    reg [7:0] s_4a53;
    reg [7:0] v_6288[511:0];
    initial begin
    end
    reg [7:0] s_2fcd;
    
    always @ (posedge pa_clk ) begin : PROC0
        if ((pa_we == 1'b1)) begin
            v_6288[pa_wa] <= pa_wd;
        end
        s_4a53 <= v_6288[pa_ra];
    end
    assign  pa_rd = s_4a53;
    
    always @ 

# TDPRAM verification

TDPRAM verification can be done on various levels:

* Functional verification without collision detection via CXXRTL
* HDL based model simulation against a Verilog or VHDL primitive vendor model with collision detection
* Post-map simulation after mapping using a synthesis step

The fine detail in the above HDL models: depending on the RAM instance (as VHDL shared variable or a Verilog signal), as well as the specific assignment mode (blocking or non-blocking) in the Verilog inference do have a **crucial impact** on the read/write priorities of such RAMs.

Most TDP RAM models allow the following priority modes.
* TRANSPARENCY: When a value is written on a port, it is available on the data output on the next clock cycle
* READ_BEFORE_WRITE: The previous value of the memory cell at the given address is available on the output on the next clock cycle

For concurrent accesses from two ports, even more complexity is introduced. For example, a WRITE action on one port and a READ on the other  from the same address can also be subject to the priority configuration, moreover, different clock domains are getting into the play.

## Different data I/O widths

Several combinations of input and output port widths are supported by TDPRAM architectures in general. If both data widths are in an integer ratio, only address translation and mapping is relevant for either bitwise slicing or concatenation. If not, alignment rules may apply.

As these properties are very technology dependent, it turns out to be complex to generalize test routines.
This is addressed by a generic `tdp_tester` base class, containing a few common test routines. By derivation, customized TDP RAM testers are created that run a customized test for a particular architecture.

**Note** This may require a particular installation of the cyrite tdpram library

In [12]:
from cyrite.library.ram.tdpram import TDPRamGenerator

from myirl.test.ghdl import GHDL93
from myirl.test.icarus import ICARUS

class my_tester(TDPRamGenerator):
    def __init__(self, SIMULATOR = ICARUS, delta = 0.2,
        name = "tb_priorities"):
        SIZE = 320*64

        super().__init__(name, SIMULATOR, size = SIZE)

        # Enable strict transparency check
        self.STRICT_XTRANSPARENCY_CHECK = True
        # Use generic wrapper as Unit Under Test:
        self.uut = self.tdpram_wrapper
        self.MODE_RAW = 'NORMAL'
        self.MODE_WT = 'WRITETHROUGH'
        self.undefined = Undefined


## Priority tests

The `tb_priorities` test uses two particularely detuned clocks for each RAM port and a small DELTA window for coincidence checks. It verifies that the RAM model reports an `Undefined` signal value when simultaneous writes/reads occur in `WRITETHROUGH` (Transparency) mode.

In [13]:

d = my_tester()
tb = d.tb_priorities(WMODE = "WRITETHROUGH")

tb.run(2000, debug = True, wavetrace = "test_ghdl.vcd")


 Declare obj 'tb_priorities' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
 DSIZE: use default 16 
 ASIZE: use default 10 
 CLKA_PERIOD: use default 3.02 
 CLKB_PERIOD: use default 3.61 
 OUT_REG: use default False 
 Declare obj 'trigger_pulse' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
 Declare obj 'coincidence_prio' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
 DELTA: use default 0.2 
 Declare obj 'clkpulse' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
DEBUG: INSTANCING uut with wmode = WRITETHROUGH
 Declare obj 'tdpram_wrapper' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
 Declare obj 'tdpram' in context '(my_tester 'tb_priorities')'(<class '__main__.my_tester'>) 
 INITDATA: use default None 
 DELTA: use default 0.2 
 DELTA: use default 0.2 
 DELTA: use default 0.2 
 Writing 'gray_counter' to file /tmp/gray_counter.v 
DEBUG Fallback wire for r
 Writ

/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg WMODE (<class 'str'>)
  base.warnings.warn(msg)
/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg DELTA (<class 'float'>)
  base.warnings.warn(msg)
/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/kernel/components.py:118: UserWarning: Fallback: Pass through other argument for arg wmode (<class 'str'>)
  base.warnings.warn(msg)
DEBUG IGNORING SET INIT FOR CLKSIGNAL
DEBUG IGNORING SET INIT FOR CLKSIGNAL
/home/cyrite/.local/lib/python3.10/site-packages/cyritehdl-0.1b0-py3.10-linux-x86_64.egg/myirl/test/icarus.py:40: UserWarning: Ignoring wavetrace argument for Verilog simulator
  warnings.warn("Ignoring wavetrace argument for Verilog simu

0

In [14]:
!cat /tmp/tb_priorities.v

// File generated from source:
//     <not available (cython)>
// (c) 2016-2022 section5.ch
// Modifications may be lost, edit the source file instead.

`timescale 1 ns / 1 ps
`include "aux.v"
// Architecture irl_uncached



module tb_priorities
    ();
    // Local type declarations
    // Signal declarations
    reg /* std_ulogic */ a_clk;
    initial a_clk = 1'b0;
    reg /* std_ulogic */ b_clk;
    initial b_clk = 1'b0;
    wire /* std_ulogic */ pa;
    wire /* std_ulogic */ pb;
    wire /* std_ulogic */ a_is_first;
    wire /* std_ulogic */ coincidence_reg;
    wire /* std_ulogic */ co_en;
    wire /* std_ulogic */ coincidence;
    wire [3:0] gca;
    wire /* std_ulogic */ fill_en;
    reg /* std_ulogic */ r;
    initial r = 1'b0;
    wire [3:0] gcb;
    wire /* std_ulogic */ valid;
    reg /* std_ulogic */ stat_raw;
    reg /* std_ulogic */ stat_rbw;
    reg /* std_ulogic */ stat_wt;
    reg /* std_ulogic */ stat_wto;
    reg /* std_ulogic */ stat_coincidence;
    reg /* std_ulog